# Correct_parameter_choosing

Due to a problem in original scripts, the min_dist parameter was been reading as `int` and not `float` , and therefore for some groups of documents some visualizations were not made. The missing visualizations are:

In [1]:
# Checking Visualizations

# EU - Daten und KI ## FEITO

#  (5, 0.25, 30, 7),
#  (10, 0.0, 10, 2), OK
#  (10, 0.0, 15, 3), OK
#  (10, 0.0, 20, 5), OK
#  (10, 0.1, 10, 2),
#  (10, 0.1, 15, 3),
#  (10, 0.1, 20, 5),
#  (10, 0.1, 25, 6),
#  (10, 0.25, 10, 2),
#  (10, 0.25, 20, 5),
#  (10, 0.25, 25, 6),  
#  (20, 0.0, 15, 3), OK
#  (20, 0.1, 15, 3),
#  (20, 0.1, 20, 5),
#  (20, 0.25, 15, 3),
#  (20, 0.25, 30, 7)


# EU - Timber

# (5.0, 0.0, 10, 2) OK
# (7.0, 0.1, 30, 7)
# (7.0, 0.25, 25, 6)
# (7.0, 0.25, 30, 7)
# (10.0, 0.0, 25, 6) OK
# (10.0, 0.1, 25, 6)
# (20.0, 0.25, 10, 2)

# AT Daten KI

# (7.0, 0.25, 30, 7) OK
# (10.0, 0.0, 30, 7) OK
# (10.0, 0.1, 25, 6) OK
# (10.0, 0.25, 25, 6) OK
# (20.0, 0.0, 25, 6) OK
# (20.0, 0.0, 30, 7) OK
# (20.0, 0.25, 25, 6) OK
# (20.0, 0.25, 30, 7) OK

# AT Regierung
# (5.0, 0.25, 25, 6)
# (7.0, 0.25, 20, 5)
# (7.0, 0.25, 25, 6)
# (7.0, 0.25, 30, 7)
# (20.0, 0.0, 20, 5) OK

# AT timber

# (5.0, 0.0, 10, 2) OK
# (5.0, 0.0, 15, 3) OK
# (5.0, 0.1, 15, 3)
# (5.0, 0.25, 10, 2)
# (5.0, 0.25, 20, 5)
# (7.0, 0.1, 10, 2)
# (7.0, 0.25, 10, 2)
# (10.0, 0.1, 10, 2)
# (10.0, 0.25, 10, 2)
# (10.0, 0.25, 15, 3)
# (20.0, 0.1, 10, 2)
# (20.0, 0.1, 15, 3)
# (20.0, 0.25, 10, 2)
# (20.0, 0.25, 15, 3)

# DE Regierung

# (5.0, 0.0, 15, 3) OK
# (5.0, 0.1, 10, 2) OK
# (5.0, 0.1, 15, 3) OK
# (7.0, 0.0, 10, 2) OK

# DE Timber

# (7.0, 0.0, 15, 3) OK
# (7.0, 0.0, 25, 6) OK
# (7.0, 0.1, 20, 5) OK
# (7.0, 0.25, 20, 5) OK
# (7.0, 0.25, 25, 6) OK
# (10.0, 0.0, 20, 5) OK
# (10.0, 0.1, 25, 6) OK
# (10.0, 0.25, 25, 6) OK
# (20.0, 0.0, 25, 6) OK

In [2]:
## Importando tudo que já está fixo:

# Data processing
import json
import numpy as np
import pandas as pd
import re


# Topic model
import umap
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN

# Dimension reduction
from umap import UMAP


# Countvectorizer (para stopwords)
from sklearn.feature_extraction.text import CountVectorizer

## For UMAP Analisys:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline
sns.set()


from sentence_transformers import SentenceTransformer

# para montar as combinacoes:
from itertools import product

# Para a matrix 
from sklearn.metrics.pairwise import cosine_similarity

# Topic Coherence:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

## Functions to clean data:

def replace_char(x):
  return re.sub('[^a-zA-Z\süöäß]', '', x)


def clean_corpus(s):
    s = s.apply(lambda x: x.lower() if isinstance(x, str) else x)     # tolower:
    idx = s.apply(lambda x: isinstance(x, str))                       # Criar um índice booleano para filtrar os valores não-texto
    s = s.drop(idx[~idx].index)                                       # Remover os valores não-texto
    s = s.apply(replace_char)                                         # remover as partes de palavras que nao sao caracteres ou espacos
    idx = s.apply(lambda x: len(x) < 2)                               # Criar um índice booleano para filtrar os elementos com comprimento menor que 2
    s = s.drop(idx[idx].index)                                        # Remover os elementos com comprimento menor que 2
    idx = s.apply(lambda x: x == '')                                  # Criar um índice booleano para filtrar os elementos vazios
    s = s.drop(idx[idx].index)                                        # Remover os elementos vazios
    
    return s

def calculuate_coherence_score(topic_model):  
  #variable
  topic_words = topic_words = [[words for words, _ in topic_model.get_topic(topic) if words!=''] 
               for topic in range(len(set(topics))-1)]
  vectorizer = topic_model.vectorizer_model
  tokenizer = vectorizer.build_tokenizer()
  #dictionary
  tokens = [doc.split() for doc in docs]
  dictionary = corpora.Dictionary(tokens)
  corpus = [dictionary.doc2bow(token) for token in tokens]


  coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
  coherence = coherence_model.get_coherence()

  
  return coherence



c:\python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# EU - Daten und KI

combinations = [ (5, 0.25, 30, 7), (10, 0.1, 10, 2),  (10, 0.1, 15, 3),  (10, 0.1, 20, 5),  (10, 0.1, 25, 6),  (10, 0.25, 10, 2),  (10, 0.25, 20, 5),
                (10, 0.25, 25, 6),  (20, 0.1, 15, 3), (20, 0.1, 20, 5),  (20, 0.25, 15, 3),  (20, 0.25, 30, 7)]

# Abrindo o arquivos
df_original = pd.read_csv("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champions_2.0\\01_gesetze_sammlung\\df_data_ki.csv")


# German stopwords que eu alterei:
# Carregue a lista a partir do arquivo
with open("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champions_2.0\\01_gesetze_sammlung\\german_stopwords.json", "r") as f:
    german_stopwords_alterada = json.load(f) 

df_work = df_original

# nome dos dados:
tinicio = 'eu_daten_ki_'

docs = df_original['text']
docs = clean_corpus(docs).to_list()



###### inicio do loop #######



df2 = pd.DataFrame({'Document': [],'Topic': [], 'Name': [], 'Top_n_words' : [], 'Probability': [], 'Representative_document': [], 'model': []})

for comb in combinations:
    print(comb)


    n_neighbors = int(comb[0])
    min_dist = float(comb[1])
    min_cluster = int(comb[2])
    min_sample = int(comb[3])


    umap_model  = UMAP(n_neighbors=n_neighbors, 
                            min_dist = min_dist,
                            n_components=5, 
                            metric='cosine', random_state=42)


    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster, 
                            min_samples = min_sample, 
                            metric='euclidean', 
                            prediction_data=True)

    vectorizer_model = CountVectorizer(stop_words=german_stopwords_alterada) #german_stop_words alterada

    topic_model = BERTopic(language = "german", vectorizer_model=vectorizer_model, umap_model=umap_model, hdbscan_model=hdbscan_model).fit(docs)

    topics, probs = topic_model.fit_transform(docs)

    # Dataframe with representative examples:

    df_temp = topic_model.get_document_info(docs)
    df_temp = df_temp.query('Representative_document == True')

    #confirmando o modelo desse df
    position = combinations.index(comb)
    position = str(position)

    #add modelo
    df_temp['model'] =  str(comb)

    df2 = pd.concat([df2, df_temp], axis=0)

        ##### Visualizacao dos resultados

    title = tinicio + '_n_neighbors = ' + str(n_neighbors) + ' min_dist = ' + str(min_dist) + ' min_cluster = ' + str(min_cluster) + ' min_sample = ' + str(min_sample)

    fig_1 = topic_model.visualize_heatmap()
    fig_1.update_layout(title = title)

    ############################################Visualizacao da distribuicao

    sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")   #Suitable Score Functions: cosine
    embeddings = sentence_model.encode(docs, show_progress_bar=False)


    # Train BERTopic
    topic_model = BERTopic(language = "german", 
                            umap_model=umap_model,
                            vectorizer_model=vectorizer_model,
                            hdbscan_model=hdbscan_model).fit(docs, embeddings)


    # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
    reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)
    fig_2 = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)
    fig_2.update_layout(title = title)

    title = re.sub(r'\s', '_', title) + '.html'

    with open(title, 'a') as f:
        f.write(fig_1.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write(fig_2.to_html(full_html=False, include_plotlyjs='cdn'))

#Fora do loop:

tituloex_models = 'extra_df_examples_models_' + tinicio + '.csv'
df2.to_csv(tituloex_models, sep = ';', index = False, )

(5, 0.25, 30, 7)
(10, 0.1, 10, 2)
(10, 0.1, 15, 3)
(10, 0.1, 20, 5)
(10, 0.1, 25, 6)
(10, 0.25, 10, 2)
(10, 0.25, 20, 5)
(10, 0.25, 25, 6)
(20, 0.1, 15, 3)
(20, 0.1, 20, 5)
(20, 0.25, 15, 3)
(20, 0.25, 30, 7)


In [4]:
# EU - Timber

combinations = [(7.0, 0.1, 30, 7), (7.0, 0.25, 25, 6), (7.0, 0.25, 30, 7), (10.0, 0.1, 25, 6), (20.0, 0.25, 10, 2)]

# Abrindo o arquivos
df_clima = pd.read_csv("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champI4.0ns\\gesetze_sammlung\\df_clima.csv")
df_timber = pd.read_csv("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champI4.0ns\\gesetze_sammlung\\df_timber.csv")

# Ajeitando os dados:

df_original = pd.concat([df_clima, df_timber])

# German stopwords que eu alterei:
# Carregue a lista a partir do arquivo
with open("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champions_2.0\\01_gesetze_sammlung\\german_stopwords.json", "r") as f:
    german_stopwords_alterada = json.load(f) 

df_work = df_original

# nome dos dados:
tinicio = 'eu_timber_'

docs = df_work['text']
docs = clean_corpus(docs).to_list()



###### inicio do loop #######



df2 = pd.DataFrame({'Document': [],'Topic': [], 'Name': [], 'Top_n_words' : [], 'Probability': [], 'Representative_document': [], 'model': []})

for comb in combinations:
    print(comb)


    n_neighbors = int(comb[0])
    min_dist = float(comb[1])
    min_cluster = int(comb[2])
    min_sample = int(comb[3])


    umap_model  = UMAP(n_neighbors=n_neighbors, 
                            min_dist = min_dist,
                            n_components=5, 
                            metric='cosine', random_state=42)


    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster, 
                            min_samples = min_sample, 
                            metric='euclidean', 
                            prediction_data=True)

    vectorizer_model = CountVectorizer(stop_words=german_stopwords_alterada) #german_stop_words alterada

    topic_model = BERTopic(language = "german", vectorizer_model=vectorizer_model, umap_model=umap_model, hdbscan_model=hdbscan_model).fit(docs)

    topics, probs = topic_model.fit_transform(docs)

    # Dataframe with representative examples:

    df_temp = topic_model.get_document_info(docs)
    df_temp = df_temp.query('Representative_document == True')

    #confirmando o modelo desse df
    position = combinations.index(comb)
    position = str(position)

    #add modelo
    df_temp['model'] =  str(comb)

    df2 = pd.concat([df2, df_temp], axis=0)

        ##### Visualizacao dos resultados

    title = tinicio + '_n_neighbors = ' + str(n_neighbors) + ' min_dist = ' + str(min_dist) + ' min_cluster = ' + str(min_cluster) + ' min_sample = ' + str(min_sample)

    fig_1 = topic_model.visualize_heatmap()
    fig_1.update_layout(title = title)

    ############################################Visualizacao da distribuicao

    sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")   #Suitable Score Functions: cosine
    embeddings = sentence_model.encode(docs, show_progress_bar=False)


    # Train BERTopic
    topic_model = BERTopic(language = "german", 
                            umap_model=umap_model,
                            vectorizer_model=vectorizer_model,
                            hdbscan_model=hdbscan_model).fit(docs, embeddings)


    # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
    reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)
    fig_2 = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)
    fig_2.update_layout(title = title)

    title = re.sub(r'\s', '_', title) + '.html'

    with open(title, 'a') as f:
        f.write(fig_1.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write(fig_2.to_html(full_html=False, include_plotlyjs='cdn'))

#Fora do loop:

tituloex_models = 'extra_df_examples_models_' + tinicio + '.csv'
df2.to_csv(tituloex_models, sep = ';', index = False, )

(7.0, 0.1, 30, 7)
(7.0, 0.25, 25, 6)
(7.0, 0.25, 30, 7)
(10.0, 0.1, 25, 6)
(20.0, 0.25, 10, 2)


In [ ]:
##### Continuar daqui:

In [5]:
# AT Regierung
combinations = [(5.0, 0.25, 25, 6), (7.0, 0.25, 20, 5), (7.0, 0.25, 25, 6), (7.0, 0.25, 30, 7)]

# Abrindo o arquivos
df_original = pd.read_json("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champions_2.0\\01_gesetze_sammlung\\at_concatenado.json")

# Abrindo o arquivos
df_work = df_original.query("documento == 'regierungsprogramm'")

# Austrian stopwords que eu alterei:
# Carregue a lista a partir do arquivo
with open("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champions_2.0\\01_gesetze_sammlung\\austrian_stopwords.json", "r") as f:
    german_stopwords_alterada = json.load(f) 


# nome dos dados:
tinicio = 'at_regierung_'

docs = df_work['text']
docs = clean_corpus(docs).to_list()



###### inicio do loop #######



df2 = pd.DataFrame({'Document': [],'Topic': [], 'Name': [], 'Top_n_words' : [], 'Probability': [], 'Representative_document': [], 'model': []})

for comb in combinations:
    print(comb)


    n_neighbors = int(comb[0])
    min_dist = float(comb[1])
    min_cluster = int(comb[2])
    min_sample = int(comb[3])


    umap_model  = UMAP(n_neighbors=n_neighbors, 
                            min_dist = min_dist,
                            n_components=5, 
                            metric='cosine', random_state=42)


    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster, 
                            min_samples = min_sample, 
                            metric='euclidean', 
                            prediction_data=True)

    vectorizer_model = CountVectorizer(stop_words=german_stopwords_alterada) #german_stop_words alterada

    topic_model = BERTopic(language = "german", vectorizer_model=vectorizer_model, umap_model=umap_model, hdbscan_model=hdbscan_model).fit(docs)

    topics, probs = topic_model.fit_transform(docs)

    # Dataframe with representative examples:

    df_temp = topic_model.get_document_info(docs)
    df_temp = df_temp.query('Representative_document == True')

    #confirmando o modelo desse df
    position = combinations.index(comb)
    position = str(position)

    #add modelo
    df_temp['model'] =  str(comb)

    df2 = pd.concat([df2, df_temp], axis=0)

        ##### Visualizacao dos resultados

    title = tinicio + '_n_neighbors = ' + str(n_neighbors) + ' min_dist = ' + str(min_dist) + ' min_cluster = ' + str(min_cluster) + ' min_sample = ' + str(min_sample)

    fig_1 = topic_model.visualize_heatmap()
    fig_1.update_layout(title = title)

    ############################################Visualizacao da distribuicao

    sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")   #Suitable Score Functions: cosine
    embeddings = sentence_model.encode(docs, show_progress_bar=False)


    # Train BERTopic
    topic_model = BERTopic(language = "german", 
                            umap_model=umap_model,
                            vectorizer_model=vectorizer_model,
                            hdbscan_model=hdbscan_model).fit(docs, embeddings)


    # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
    reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)
    fig_2 = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)
    fig_2.update_layout(title = title)

    title = re.sub(r'\s', '_', title) + '.html'

    with open(title, 'a') as f:
        f.write(fig_1.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write(fig_2.to_html(full_html=False, include_plotlyjs='cdn'))

#Fora do loop:

tituloex_models = 'extra_df_examples_models_' + tinicio + '.csv'
df2.to_csv(tituloex_models, sep = ';', index = False, )

KeyError: 'text'

In [ ]:
# AT timber

combinations = [(5.0, 0.1, 15, 3), (5.0, 0.25, 10, 2) , (5.0, 0.25, 20, 5), (7.0, 0.1, 10, 2), (7.0, 0.25, 10, 2) , (10.0, 0.1, 10, 2) , (10.0, 0.25, 10, 2), (10.0, 0.25, 15, 3), (20.0, 0.1, 10, 2), (20.0, 0.1, 15, 3), (20.0, 0.25, 10, 2), (20.0, 0.25, 15, 3)]

# Abrindo o arquivos
df_original = pd.read_json("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champions_2.0\\01_gesetze_sammlung\\at_concatenado.json")

# Abrindo o arquivos
df_work = df_original.query("documento == 'oestereiche_holzinitiative' | documento == 'nationale_strategie_kreislaufwirtschaft' ")

# Austrian stopwords que eu alterei:
# Carregue a lista a partir do arquivo
with open("C:\\Users\\JVoigt\\OneDrive - Universität für Weiterbildung Krems\\Dokumente\\Python Scripts\\champions_2.0\\01_gesetze_sammlung\\austrian_stopwords.json", "r") as f:
    german_stopwords_alterada = json.load(f) 


# nome dos dados:
tinicio = 'at_timber_'

docs = df_work['text']
docs = clean_corpus(docs).to_list()



###### inicio do loop #######



df2 = pd.DataFrame({'Document': [],'Topic': [], 'Name': [], 'Top_n_words' : [], 'Probability': [], 'Representative_document': [], 'model': []})

for comb in combinations:
    print(comb)


    n_neighbors = int(comb[0])
    min_dist = float(comb[1])
    min_cluster = int(comb[2])
    min_sample = int(comb[3])


    umap_model  = UMAP(n_neighbors=n_neighbors, 
                            min_dist = min_dist,
                            n_components=5, 
                            metric='cosine', random_state=42)


    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster, 
                            min_samples = min_sample, 
                            metric='euclidean', 
                            prediction_data=True)

    vectorizer_model = CountVectorizer(stop_words=german_stopwords_alterada) #german_stop_words alterada

    topic_model = BERTopic(language = "german", vectorizer_model=vectorizer_model, umap_model=umap_model, hdbscan_model=hdbscan_model).fit(docs)

    topics, probs = topic_model.fit_transform(docs)

    # Dataframe with representative examples:

    df_temp = topic_model.get_document_info(docs)
    df_temp = df_temp.query('Representative_document == True')

    #confirmando o modelo desse df
    position = combinations.index(comb)
    position = str(position)

    #add modelo
    df_temp['model'] =  str(comb)

    df2 = pd.concat([df2, df_temp], axis=0)

        ##### Visualizacao dos resultados

    title = tinicio + '_n_neighbors = ' + str(n_neighbors) + ' min_dist = ' + str(min_dist) + ' min_cluster = ' + str(min_cluster) + ' min_sample = ' + str(min_sample)

    fig_1 = topic_model.visualize_heatmap()
    fig_1.update_layout(title = title)

    ############################################Visualizacao da distribuicao

    sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")   #Suitable Score Functions: cosine
    embeddings = sentence_model.encode(docs, show_progress_bar=False)


    # Train BERTopic
    topic_model = BERTopic(language = "german", 
                            umap_model=umap_model,
                            vectorizer_model=vectorizer_model,
                            hdbscan_model=hdbscan_model).fit(docs, embeddings)


    # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
    reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)
    fig_2 = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)
    fig_2.update_layout(title = title)

    title = re.sub(r'\s', '_', title) + '.html'

    with open(title, 'a') as f:
        f.write(fig_1.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write(fig_2.to_html(full_html=False, include_plotlyjs='cdn'))

#Fora do loop:

tituloex_models = 'extra_df_examples_models_' + tinicio + '.csv'
df2.to_csv(tituloex_models, sep = ';', index = False, )